# TODO

https://towardsdatascience.com/winning-blackjack-using-machine-learning-681d924f197c

1. Andmestiku puhastus, ainult need asjad mida vaja on.
2. Sorteerida ära terve andmestiku peale pärast esimese kahe kaardi andmist kas käsi on soft-hand, hard-hand või paar (nagu need tabelid üleval antud lehel on). Hard-handid saame nii kätte, et kus kaks kaarti pole kas paar või üks kahest kaardist ei ole 
3. Nüüd kui meil on ära sorteeritud käed, siis iga käega peaksime välja arvutama (I assume) võiduprotsendi järgi kas me peame hittima, stay või panust duubeldama. 
4. Ja nii arvutame välja kõikide juhtudega ehk kui sul on kolm kaarti käes, neli kaarti jne.

In [16]:
import pandas as pd
import warnings

data = pd.read_csv("blkjckhands.csv", index_col=False)
data = data.drop(columns=['Unnamed: 0', 'PlayerNo', 'dealcard2', 'dealcard3', 'dealcard4', 'dealcard5', 'plybustbeat', 'dlbustbeat','plwinamt', 'dlwinamt'])


In [17]:
cond = (data.card3 == 0) & (data.card4 != 0)
data.loc[cond, ['card3', 'card4']] = data.loc[cond,['card4', 'card3']].values



#154980 - kus on kolmas kaart on õiges kohas
#190401 - kus on kolmas kaart vales kohas
#35603 - kus on kolmas ja neljas kaart vales kohas

In [18]:
cond2 = (data.card3 != 0) & (data.card4 == 0) & (data.card5 != 0)
data.loc[cond2, ['card4', 'card5']] = data.loc[cond2, ['card5', 'card4']].values

data[(data["card3"] != 0) & (data["card4"] == 0) & (data["card5"] != 0)]

,card1,card2,card3,card4,card5,sumofcards,dealcard1,sumofdeal,blkjck,winloss,ply2cardsum


In [20]:
#Sorteerisin käed vastavalt esimese kahe kaardi järgi, sealt vastavalt saame teha maatrixi siis otsustest based on win rate
#Lisaks peab jälgima seda, et meil on andmeid kus on kolmas kaart 0 aga neljas ja viies on hit. Need peame lugema pmst hittideks,
#lihtsalt 

#Ehk siis me arvutame iga käe summa järgi ning kas sellega võideti või mitte. Lisaks vaatame dealeri kaarti. Umbes kuidagi selline on
#arvutus:
# 1. Näiteks hard_hands puhul võtame summa nagu näiteks 10. 
# 2. Loeme kõik 10ned kokku. Eraldame kõik diileri kaardi juhud ning võtame näiteks kus on sinu kaartide summa on 10 ning diileri summa on 2
# 3. Loeme kokku win% kõikidel kätel kus hitidakse ning kus ei hitida. (not sure how to make this work yet).
# 4. Paneme sinu tehtud maatriksisse lõpuks 0 asemele kas me hitime selle summaga ning selle dealeri handiga või standime.

#Sarnast algoritmi saab kasutada ka paaride ning soft_handide korral ehk me võime kasutada paaride korral näiteks alguses nn paaride maatriksit,
#ning näiteks teeme statistika selle põhjal kas splittida või mitte. Sealt me edasi mõtlema ei pea, kuna edasi läheb ta nkni otsustab kas tema käsi
#on hard_hand või mis iganes. Sama ka soft_handide põhjal.

pairs = data[data["card1"] == data["card2"]].copy()
soft_hands = data[(data["card1"] == 1) | (data["card1"] == 11) | (data["card2"] == 1) | (data["card2"] == 11)].copy()
hard_hands = data[(data["card1"] != data["card2"]) & ((data["card1"] != 1) & (data["card1"] != 11) & (data["card2"] != 1) & (data["card2"] != 11))].copy()

pairs[(pairs["card1"] == 2)]

,card1,card2,card3,card4,card5,sumofcards,dealcard1,sumofdeal,blkjck,winloss,ply2cardsum
73,2,2,4,3,10,21,9,19,nowin,Win,4
83,2,2,10,1,10,25,7,17,nowin,Loss,4
121,2,2,11,7,0,22,10,19,nowin,Loss,4
331,2,2,6,7,0,17,10,20,nowin,Loss,4
451,2,2,10,0,0,14,5,18,nowin,Loss,4
...,...,...,...,...,...,...,...,...,...,...,...
899454,2,2,2,9,0,15,5,21,nowin,Loss,4
899493,2,2,11,0,0,15,4,23,nowin,Win,4
899736,2,2,8,1,0,13,3,20,nowin,Loss,4
899839,2,2,10,0,0,14,5,17,nowin,Loss,4


Otsustuse dataframe on vaja panna paika siis nende kriteerimite alusel:
1. Row tähistab handi väärtust ja veerg tähistab dealeri esimest kaarti. Me peame row väärtuseks võtma ka need handid kus ntks alles viies kaart oli see mis summaks andis 20 ehk me itereerime iga rida kus ka kaartide vaheväärtus oli otsitav summa. Ühesõnaga ütleme, et me otsime hande kus kaartide summaks oli 18, st et me vaatame hande kus esimese kahe kaardi, esimese kolme kaari, esimese nelja kaardi... jne summa oli 18 ja kas siis oli HIT v oli STAND ja kas käsi lõpuks võideti või ei.

2. Me loeme HITID ja STANDID kokku ja vaatame kumma puhul suurem võidu % oli ja kanname vastavalt siis kummagi tabelisse.

In [21]:
veerud = {'2':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '3':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '4':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '5':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '6':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '7':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '8':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         '9':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         'T':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0'],
         'A':['0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0']}
hardHands = pd.DataFrame(veerud, index=['20','19','18','17','16','15','14','13','12','11','10','9','8','7','6','5'])

In [22]:
veerud = {'2':['0','0','0','0','0','0','0','0'],
         '3':['0','0','0','0','0','0','0','0'],
         '4':['0','0','0','0','0','0','0','0'],
         '5':['0','0','0','0','0','0','0','0'],
         '6':['0','0','0','0','0','0','0','0'],
         '7':['0','0','0','0','0','0','0','0'],
         '8':['0','0','0','0','0','0','0','0'],
         '9':['0','0','0','0','0','0','0','0'],
         'T':['0','0','0','0','0','0','0','0'],
         'A':['0','0','0','0','0','0','0','0']}

softHands = pd.DataFrame(veerud, index=['A-9', 'A-8', 'A-7', 'A-6', 'A-5', 'A-4', 'A-3', 'A-2'])
pairTable = pd.DataFrame(veerud, index=['A-A', 'T-T', '9-9', '8-8', '7-7', '6-6', '5-5', '4-4','3-3','2-2'])

In [30]:
#Teeme ühe korra arvutuse läbi
#Võtame kaartide summaks 5 ja dealeri uphandiks 2
#temp = hard_hands[(hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] + hard_hands["card4"] + hard_hands["card5"] == 5)].copy()
def hitvstand(summa,dcard, hand):
    if(hand == 'hard'):
        summa = int(summa)
        if(dcard == "A"):
            temp = hard_hands[
                ((hard_hands["ply2cardsum"] == summa)| (hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] == summa)
                |(hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] + hard_hands["card4"] == summa))
                 & ((hard_hands["dealcard1"] == 1) | (hard_hands["dealcard1"] == 11))
                ].copy()
        elif(dcard == "T"):
            temp = hard_hands[
                ((hard_hands["ply2cardsum"] == summa)| (hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] == summa)
                |(hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] + hard_hands["card4"] == summa))
                 & (hard_hands["dealcard1"] == 10)
                ].copy()
        else:
            temp = hard_hands[
                ((hard_hands["ply2cardsum"] == summa)| (hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] == summa)
                |(hard_hands["card1"] + hard_hands["card2"] + hard_hands["card3"] + hard_hands["card4"] == summa))
                 & (hard_hands["dealcard1"] == int(dcard))
                ].copy()


        standwin = temp[(temp["winloss"]=="Win") & ((temp["card1"] + temp["card2"]==summa) & (temp["card3"]==0))
                      |
                    ((temp["card1"]+temp["card2"]+temp["card3"]==summa) & (temp["card3"]!=0) & (temp["card4"]==0))
                     |
                     ((temp["card1"]+temp["card2"]+temp["card3"]+temp["card4"]==summa) & (temp["card3"]!=0) & (temp["card4"]!=0)& (temp["card5"]==0))].copy()

        hitwin = temp[(temp["winloss"]=="Win") & ((temp["card1"] + temp["card2"]==summa) & (temp["card3"]!=0))
                      |
                    ((temp["card1"]+temp["card2"]+temp["card3"]==summa) & (temp["card3"]!=0) & (temp["card4"]!=0))
                     |
                     ((temp["card1"]+temp["card2"]+temp["card3"]+temp["card4"]==summa) & (temp["card3"]!=0) & (temp["card4"]!=0)& (temp["card5"]!=0))].copy()
        hitwinNr = len(hitwin.index)
        standwinNr = len(standwin.index)
    
    if(hand == 'soft'):
        teinekaart = summa.split("-")
        summa1 = 1 + int(teinekaart[1])
        summa2 = 11 + int(teinekaart[1])
        
        if(dcard == "A"):
            temp = soft_hands[
                ((soft_hands["ply2cardsum"] == summa1) | (soft_hands["ply2cardsum"] == summa2))
                & ((soft_hands["dealcard1"] == 1) | (soft_hands["dealcard1"] == 11))
            ].copy()
        elif(dcard == "T"):
            temp = soft_hands[
                ((soft_hands["ply2cardsum"] == summa1) | (soft_hands["ply2cardsum"] == summa2))
                & (soft_hands["dealcard1"] == 10)
            ].copy()
        else:
            temp = soft_hands[
                ((soft_hands["ply2cardsum"] == summa1) | (soft_hands["ply2cardsum"] == summa2))
                & (soft_hands["dealcard1"] == int(dcard))
            ].copy()
            
        hitwin1 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa1)
                      &
                      ((temp["card3"] != 0) & (temp["card4"] == 0) & (temp["card5"] == 0)) 
                     ].copy()
        standwin1 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa1)
                        &
                        (temp["card3"] == 0)
                       ].copy()
        hitwin2 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa2)
                      &
                      ((temp["card3"] != 0) & (temp["card4"] == 0) & (temp["card5"] == 0)) 
                     ].copy()
        standwin2 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa2)
                        &
                        (temp["card3"] == 0)
                       ].copy()
        hitwinNr = max(len(hitwin1.index), len(hitwin2.index))
        standwinNr = max(len(standwin1.index), len(standwin2.index))
        

    if(hitwinNr > standwinNr):
        return 'H'
    else:
        return 'S'
    
    if(hand=="pair"):
        card1 = summa[2]
        card2 = summa[0]
        if(card1=="A" & card2=="A"):
            card11=1
            card12=11
            card21=1
            card22=11
            summa1=2
            summa2=12
            if(dcard == "A"):
                temp = pairs[
                    (((pairs["ply2cardsum"] == summa1)|(pairs["ply2cardsum"]==summa2) &
                      ((pairs["card1"]==11)&(pairs["card2"]==1))|(pairs["card1"]==1)&(pairs["card2"]==11))
                    & ((pairs["dealcard1"] == 1) | (pairs["dealcard1"] == 11)))].copy()
            elif(dcard == "T"):
                temp = pairs[
                    (((pairs["ply2cardsum"] == summa1)|(pairs["ply2cardsum"]==summa2) &
                      ((pairs["card1"]==11)&(pairs["card2"]==1))|(pairs["card1"]==1)&(pairs["card2"]==11))
                    & (pairs["dealcard1"] == 10))
                ].copy()
            else:
                temp = pairs[
                    (((pairs["ply2cardsum"] == summa1)|(pairs["ply2cardsum"]==summa2) &
                      ((pairs["card1"]==11)&(pairs["card2"]==1))|(pairs["card1"]==1)&(pairs["card2"]==11))
                    & (pairs["dealcard1"] == int(dcard)))
                ].copy()
            
        hitwin1 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa1)
                      &
                      ((temp["card3"] != 0) & (temp["card4"] == 0) & (temp["card5"] == 0)) 
                     ].copy()
        standwin1 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa1)
                        &
                        (temp["card3"] == 0)
                       ].copy()
        hitwin2 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa2)
                      &
                      ((temp["card3"] != 0) & (temp["card4"] == 0) & (temp["card5"] == 0)) 
                     ].copy()
        standwin2 = temp[(temp["winloss"] == "Win") & (temp["ply2cardsum"] == summa2)
                        &
                        (temp["card3"] == 0)
                       ].copy()
        
        splitwin = data[((data["winloss"]=="Win") & (data["card1"]==card11))
                        |((data["winloss"]=="Win") & (data["card1"]==card11))
                        |((data["winloss"]=="Win") & (data["card1"]==card11))
                        |((data["winloss"]=="Win") & (data["card1"]==card11))]
        
        hitwinNr = max(len(hitwin1.index), len(hitwin2.index))
        standwinNr = max(len(standwin1.index), len(standwin2.index))
        splitwinNr = len(splitwin.index)
        
        return str("HIT: "+hitwinNr+ "/n STAND:" + standwinNr +"/n SPLIT: "+splitwinNr) 
        
        
        
        else:
            if(card1=="T"& card2=="T"):
                summa=20
            else:
                summa=int(card1)+int(card2)
            if(dcard == "A"):
                temp = pairs[
                    ((pairs["ply2cardsum"] == summa) 
                    & ((pairs["dealcard1"] == 1) | (pairs["dealcard1"] == 11)))].copy()
            elif(dcard == "T"):
                temp = pairs[
                    ((pairs["ply2cardsum"] == summa)
                    & (pairs["dealcard1"] == 10))
                ].copy()
            else:
                temp = pairs[
                    ((pairs["ply2cardsum"] == summa)
                    & (pairs["dealcard1"] == int(dcard)))
                ].copy()
            
        standwin = temp[(temp["winloss"]=="Win") & ((temp["card1"] + temp["card2"]==summa) & (temp["card3"]==0))
                      |
                    ((temp["card1"]+temp["card2"]+temp["card3"]==summa) & (temp["card3"]!=0) & (temp["card4"]==0))
                     |
                     ((temp["card1"]+temp["card2"]+temp["card3"]+temp["card4"]==summa) & (temp["card3"]!=0) & (temp["card4"]!=0)& (temp["card5"]==0))].copy()

        hitwin = temp[(temp["winloss"]=="Win") & ((temp["card1"] + temp["card2"]==summa) & (temp["card3"]!=0))
                      |
                    ((temp["card1"]+temp["card2"]+temp["card3"]==summa) & (temp["card3"]!=0) & (temp["card4"]!=0))
                     |
                     ((temp["card1"]+temp["card2"]+temp["card3"]+temp["card4"]==summa) & (temp["card3"]!=0) & (temp["card4"]!=0)& (temp["card5"]!=0))].copy()
        
        splitwin = data[((data["winloss"]=="Win") & data["card1"]==card1)|]
        
        
        
        
        hitwinNr = len(hitwin.index)
        standwinNr = len(standwin.index)
        
        
        
    


SyntaxError: invalid syntax (<ipython-input-30-bd2285bf8051>, line 108)

In [27]:
Columns = list(hardHands.columns)
Rows = list(hardHands.index)
for i in range(len(Rows)):
    for j in range(len(Columns)):
        taht = hitvstand(Rows[i], Columns[j], 'hard')
        hardHands.iloc[i][j] = taht
        
hardHands

,2,3,4,5,6,7,8,9,T,A
20,S,S,S,S,S,S,S,S,S,S
19,S,S,S,S,S,S,S,S,S,S
18,S,S,S,S,S,S,S,S,S,S
17,S,S,S,S,S,S,S,S,S,S
16,S,S,S,S,S,H,H,H,H,H
15,S,S,S,S,S,H,H,H,H,H
14,S,S,S,S,S,H,H,H,H,H
13,S,S,S,S,S,H,H,H,H,H
12,H,H,S,S,S,H,H,H,H,H
11,H,H,H,H,H,H,H,H,H,H


In [28]:
Columns = list(softHands.columns)
Rows = list(softHands.index)
for i in range(len(Rows)):
    for j in range(len(Columns)):
        taht = hitvstand(Rows[i], Columns[j], "soft")
        softHands.iloc[i][j] = taht
softHands

,2,3,4,5,6,7,8,9,T,A
A-9,S,S,S,S,S,S,S,S,S,S
A-8,S,S,S,S,S,S,S,S,S,S
A-7,S,S,S,S,S,S,S,S,S,S
A-6,H,H,H,H,H,H,H,H,H,H
A-5,H,H,H,H,H,H,H,H,H,H
A-4,H,H,H,H,H,H,H,H,H,H
A-3,H,H,H,H,H,H,H,H,H,H
A-2,H,H,H,H,H,H,H,H,H,H


In [29]:
""""Columns = list(pairs.columns)
Rows = list(pairs.index)
for i in range(len(Rows)):
    for j in range(len(Columns)):
        taht = hitvstand(Rows[i], Columns[j], 'pair')
        pairs.iloc[i][j] = taht
   """     
pairs

,card1,card2,card3,card4,card5,sumofcards,dealcard1,sumofdeal,blkjck,winloss,ply2cardsum
15,10,10,0,0,0,20,4,26,nowin,Win,20
21,10,10,0,0,0,20,10,18,nowin,Win,20
22,10,10,0,0,0,20,10,18,nowin,Win,20
26,10,10,0,0,0,20,8,19,nowin,Win,20
30,6,6,9,0,0,21,3,15,nowin,Win,12
...,...,...,...,...,...,...,...,...,...,...,...
899954,10,10,0,0,0,20,7,17,nowin,Win,20
899966,10,10,0,0,0,20,11,17,nowin,Win,20
899970,10,10,0,0,0,20,6,19,nowin,Win,20
899990,10,10,0,0,0,20,10,21,nowin,Loss,20
